In [1]:
import pandas as pd
import numpy as nm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

from sklearn import preprocessing

from collections import Counter
import seaborn as sb

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"
warnings.filterwarnings('ignore')

## Dataset

In [2]:
account = pd.read_csv("ficheiros_competicao_dev/account.csv")
cards = pd.read_csv("ficheiros_competicao_dev/card_dev.csv")
clients = pd.read_csv("ficheiros_competicao_dev/client.csv")
disps = pd.read_csv("ficheiros_competicao_dev/disp.csv")
districts = pd.read_csv("ficheiros_competicao_dev/district.csv")
loans = pd.read_csv("ficheiros_competicao_dev/loan_dev.csv")
trans = pd.read_csv("ficheiros_competicao_dev/trans_dev.csv")

dataset = account.rename({'frequency' : 'frequency_account', 'date' : 'date_of_creation'}, axis=1)

dataset = dataset.merge(loans) 
dataset.drop('loan_id', inplace=True, axis=1)
dataset = dataset.rename({'date' : 'date_of_loan', 'duration' : 'duration_loan', 'payments' : 'payments_loan', 'status' : 'status_loan'}, axis=1)

dataset = dataset.merge(pd.DataFrame(trans.groupby('account_id').size(), columns=['n.of trans']), left_on='account_id', right_index=True, how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['operation']=='credit in cash'].groupby('account_id').size(), columns=['credit_cash']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['operation']=='credit card withdrawal'].groupby('account_id').size(), columns=['cc_wdw']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['operation']=='withdrawal in cash'].groupby('account_id').size(), columns=['wdw_cash']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['operation']=='collection from another bank'].groupby('account_id').size(), columns=['coll_bank']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['operation']=='remittance to another bank'].groupby('account_id').size(), columns=['rem_bank']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['type']=='withdrawal'].groupby('account_id').size(), columns=['type #1']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['type']=='withdrawal in cash'].groupby('account_id').size(), columns=['type #2']), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans[trans['type']=='credit'].groupby('account_id').size(), columns=['type #3']), right_index=True, left_on='account_id', how="outer")

dataset = dataset.merge(pd.DataFrame(trans.groupby('account_id').amount.mean().round(2)).rename(columns={'amount':'avg trans amount'}), right_index=True, left_on='account_id', how="outer")
dataset = dataset.merge(pd.DataFrame(trans.groupby('account_id').balance.mean().round(2)).rename(columns={'balance':'avg trans balance'}), right_index=True, left_on='account_id', how="outer")

# dataset = dataset.merge(disps, how="outer")

# dataset = dataset.merge(cards, how="outer")
# dataset = dataset.rename({'type' : 'card_type'}, axis=1)
# dataset.drop('disp_id', inplace=True, axis=1)

# dataset = dataset.merge(clients, left_on="client_id", right_on="client_id") 
# dataset = dataset.rename({'district_id_x' : 'district_id_account', 'district_id_y' : 'district_id_client'}, axis=1)
# dataset = dataset.merge(districts, left_on = "district_id_client", right_on="district_id")

dataset = dataset.merge(districts)


disps = disps.rename(columns={"type": "disp type"})
disps['has disponent'] = nm.where(disps['disp type'] == 'DISPONENT', True, False)
cards = cards.rename(columns={"type": "card type"})
disp_card = pd.merge(disps, cards, how="outer")

disp_card_clients = pd.merge(disp_card, clients).drop_duplicates(subset=['account_id']).drop(columns=['has disponent', 'disp type', 'disp_id'])
temp = pd.merge(disp_card_clients, pd.DataFrame(disp_card.groupby('account_id').size(), columns = ['count owner']), right_index=True, left_on='account_id', how="outer")
disp_card_clients['has disponent'] = nm.where(temp['count owner'] == 2, True, False)

dataset = dataset.merge(disp_card_clients)

dataset.to_csv("ligma")

##  Pre Processing

In [3]:
def nulls(dataset, jcolumns):
    for col in jcolumns:
        if(col in dataset.columns.tolist()):
            for row in range(0,len(dataset)):
                if dataset.iloc[row][col] is None:
                    dataset.drop(row, axis=0, inplace=True)
        else:
            print('Error: column ' + col + ' does not exist in dataset')
    dataset.fillna(0) 
    return dataset

def checknullcolumns(dataset):
    print("Number of null values:" + str(dataset.isna().sum().sum()))
    for col in dataset.columns.values.tolist():
        print("number of null rows in " + col)
        print(dataset[col].isna().sum())



In [4]:
# dataset = nulls(dataset, ['account_id','client_id', 'card_id'])

# checknullcolumns(dataset)

for col in ['cc_wdw', 'wdw_cash', 'coll_bank', 'rem_bank', 'credit_cash', 'type #1', 'type #2', 'type #3']:
    dataset[col] = dataset[col].fillna(0)

dataset = dataset.drop(columns=['issued', 'card type', 'card_id'])

checknullcolumns(dataset)

Number of null values:0
number of null rows in account_id
0
number of null rows in district_id
0
number of null rows in frequency_account
0
number of null rows in date_of_creation
0
number of null rows in date_of_loan
0
number of null rows in amount
0
number of null rows in duration_loan
0
number of null rows in payments_loan
0
number of null rows in status_loan
0
number of null rows in n.of trans
0
number of null rows in credit_cash
0
number of null rows in cc_wdw
0
number of null rows in wdw_cash
0
number of null rows in coll_bank
0
number of null rows in rem_bank
0
number of null rows in type #1
0
number of null rows in type #2
0
number of null rows in type #3
0
number of null rows in avg trans amount
0
number of null rows in avg trans balance
0
number of null rows in name
0
number of null rows in region
0
number of null rows in no. of inhabitants
0
number of null rows in no. of municipalities with inhabitants < 499
0
number of null rows in no. of municipalities with inhabitants 500

In [5]:
dataset.info

<bound method DataFrame.info of      account_id  district_id frequency_account  date_of_creation  \
0          5270         44.0  monthly issuance          930113.0   
1          5385         44.0   weekly issuance          930521.0   
2          3037         44.0  monthly issuance          940526.0   
3         10942         44.0   weekly issuance          940605.0   
4         11265         15.0  monthly issuance          930114.0   
..          ...          ...               ...               ...   
263        4462         73.0   weekly issuance          951227.0   
264        5001         53.0  monthly issuance          951215.0   
265        3084         59.0   weekly issuance          960128.0   
266       11054         59.0   weekly issuance          960201.0   
267        8680         17.0  monthly issuance          960212.0   

     date_of_loan    amount  duration_loan  payments_loan  status_loan  \
0        931122.0   79608.0           24.0         3317.0          1.0   
1  

In [6]:
dataset.dtypes

account_id                                            int64
district_id                                         float64
frequency_account                                    object
date_of_creation                                    float64
date_of_loan                                        float64
amount                                              float64
duration_loan                                       float64
payments_loan                                       float64
status_loan                                         float64
n.of trans                                            int64
credit_cash                                           int64
cc_wdw                                              float64
wdw_cash                                            float64
coll_bank                                           float64
rem_bank                                            float64
type #1                                             float64
type #2                                 

In [7]:
# Creating correlation matrix
dataset_corr = dataset.corr().abs()
dataset_corr

,account_id,district_id,date_of_creation,date_of_loan,amount,duration_loan,payments_loan,status_loan,n.of trans,credit_cash,...,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '96,client_id,birth_number,has disponent
account_id,1.000000,0.039562,0.030956,0.060458,0.023877,0.001406,0.031709,0.099614,0.070785,0.048214,...,0.005587,0.038227,0.103150,0.113278,0.050771,0.071100,0.099782,0.999886,0.104163,0.045247
district_id,0.039562,1.000000,0.024400,0.038526,0.027119,0.071698,0.027257,0.010076,0.000580,0.041173,...,0.402212,0.240566,0.185166,0.465074,0.553364,0.636465,0.522394,0.040016,0.098483,0.038792
date_of_creation,0.030956,0.024400,1.000000,0.815511,0.024643,0.068419,0.004793,0.056623,0.307901,0.212791,...,0.196947,0.046330,0.020089,0.009026,0.057849,0.062210,0.027733,0.031312,0.022628,0.004113
date_of_loan,0.060458,0.038526,0.815511,1.000000,0.101325,0.167922,0.002600,0.046012,0.160304,0.072419,...,0.157309,0.102569,0.000996,0.053961,0.088609,0.080289,0.057486,0.061229,0.044283,0.026010
amount,0.023877,0.027119,0.024643,0.101325,1.000000,0.581755,0.718617,0.092365,0.085621,0.094660,...,0.021444,0.009881,0.041666,0.000531,0.009774,0.015488,0.005835,0.023693,0.083667,0.051076
duration_loan,0.001406,0.071698,0.068419,0.167922,0.581755,1.000000,0.039838,0.049515,0.097854,0.145629,...,0.001415,0.024975,0.041896,0.061882,0.010226,0.026260,0.077691,0.001850,0.081607,0.072132
payments_loan,0.031709,0.027257,0.004793,0.002600,0.718617,0.039838,1.000000,0.166712,0.000261,0.028534,...,0.044568,0.050143,0.051555,0.000358,0.018044,0.024082,0.000700,0.030350,0.016033,0.021442
status_loan,0.099614,0.010076,0.056623,0.046012,0.092365,0.049515,0.166712,1.000000,0.104216,0.031793,...,0.062085,0.052480,0.021200,0.023856,0.024179,0.021605,0.012991,0.100180,0.043791,0.224166
n.of trans,0.070785,0.000580,0.307901,0.160304,0.085621,0.097854,0.000261,0.104216,1.000000,0.631869,...,0.051513,0.108900,0.024016,0.036815,0.023098,0.042326,0.006193,0.070924,0.043722,0.061984
credit_cash,0.048214,0.041173,0.212791,0.072419,0.094660,0.145629,0.028534,0.031793,0.631869,1.000000,...,0.061180,0.074625,0.065628,0.003542,0.006743,0.043155,0.013388,0.047934,0.048441,0.027023


In [8]:
def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x : 1 if x == positive_value else 0)
    return df
    
def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x : ordering.index(x))
    return df

def special_binary_encode(df, column, negative_value):
    df = df.copy()
    df['card_owner'] = df[column].apply(lambda x : 0 if x == negative_value else 1)
    df = df.drop(column, axis=1)
    return df
# Encode column of dataframe
def encode_column(df, column_to_encode):
    
    df_ = df.copy() # copy dataframe to avoid making changes to it inside the function
    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(df_[column_to_encode].unique())
    df_[column_to_encode] = label_encoder.transform(df_[column_to_encode])
    
    return df_

In [9]:
 # Ordinal encode the frequency column
freq_ordering = [
    'monthly issuance',
    'weekly issuance',
    'issuance after transaction'
]
dataset = ordinal_encode(dataset, 'frequency_account', ordering=freq_ordering)

# Binary encode the card column (has or not card)
freq_ordering = [
    'monthly issuance',
    'weekly issuance',
    'issuance after transaction'
]

In [10]:
def district_processing(df):

    df = encode_column(df, 'name')
    df = encode_column(df, 'region')
    df.loc[df['no. of commited crimes \'95']=="?", 'no. of commited crimes \'95'] = pd.to_numeric( df[df['no. of commited crimes \'95'] != '?']['no. of commited crimes \'95'] ).median()
    median_unemploymant_95 = (df[df['unemploymant rate \'95'] != '?']['unemploymant rate \'95']).astype(float).median()
    df.loc[df['unemploymant rate \'95']=="?", 'unemploymant rate \'95'] = median_unemploymant_95

    return df


In [11]:
dataset = district_processing(dataset)
# Encode the label (loan status) column
label_mapping = {-1: 1, 1: 0}
dataset['status_loan'] = dataset['status_loan'].replace(label_mapping)
dataset

,account_id,district_id,frequency_account,date_of_creation,date_of_loan,amount,duration_loan,payments_loan,status_loan,n.of trans,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,client_id,birth_number,has disponent
0,5270,44.0,0,930113.0,931122.0,79608.0,24.0,3317.0,0.0,53,...,53.0,8254,2.79,3.76,97,2166,2325,6367,700428,False
1,5385,44.0,1,930521.0,940407.0,149340.0,60.0,2489.0,0.0,56,...,53.0,8254,2.79,3.76,97,2166,2325,6499,630201,False
2,3037,44.0,0,940526.0,960418.0,277884.0,36.0,7719.0,1.0,117,...,53.0,8254,2.79,3.76,97,2166,2325,3670,805917,False
3,10942,44.0,1,940605.0,951002.0,314520.0,60.0,5242.0,0.0,130,...,53.0,8254,2.79,3.76,97,2166,2325,13451,556118,False
4,11265,15.0,0,930114.0,930915.0,52788.0,12.0,4399.0,0.0,43,...,51.9,9045,3.13,3.60,124,1845,1879,13845,730216,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,4462,73.0,1,951227.0,960514.0,66480.0,24.0,2770.0,1.0,10,...,56.4,8746,3.33,3.74,90,4355,4433,5384,350721,False
264,5001,53.0,0,951215.0,961109.0,110112.0,24.0,4588.0,0.0,82,...,50.9,8240,2.53,3.56,99,1850,1903,6042,631012,False
265,3084,59.0,1,960128.0,960828.0,253512.0,36.0,7042.0,1.0,29,...,62.1,8444,3.24,3.47,106,2595,2305,3730,455807,False
266,11054,59.0,1,960201.0,960820.0,148920.0,60.0,2482.0,0.0,31,...,62.1,8444,3.24,3.47,106,2595,2305,13590,770803,False


## Training

In [12]:
# Choosing the upper triangle of the correlation matrix
upper_triangle = dataset_corr.where(nm.triu(nm.ones(dataset_corr.shape), k=1).astype(bool))

# Looking for columns with correlation higher than 0.95
collumn_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]


In [13]:

# Scale X with a standard scaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)
y = dataset['status_loan'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y , train_size=0.8,random_state=123)
# labels

# y_train

In [14]:
{column: len(X[column].unique()) for column in X.columns}


{'account_id': 268,
 'district_id': 70,
 'frequency_account': 3,
 'date_of_creation': 243,
 'date_of_loan': 237,
 'amount': 258,
 'duration_loan': 5,
 'payments_loan': 249,
 'status_loan': 2,
 'n.of trans': 135,
 'credit_cash': 40,
 'cc_wdw': 6,
 'wdw_cash': 80,
 'coll_bank': 21,
 'rem_bank': 44,
 'type #1': 94,
 'type #2': 14,
 'type #3': 64,
 'avg trans amount': 268,
 'avg trans balance': 268,
 'name': 70,
 'region': 8,
 'no. of inhabitants': 70,
 'no. of municipalities with inhabitants < 499': 52,
 'no. of municipalities with inhabitants 500-1999': 33,
 'no. of municipalities with inhabitants 2000-9999': 16,
 'no. of municipalities with inhabitants >10000': 6,
 'no. of cities': 11,
 'ratio of urban inhabitants': 63,
 'average salary': 70,
 "unemploymant rate '95": 64,
 "unemploymant rate '96": 66,
 'no. of enterpreneurs per 1000 inhabitants': 44,
 "no. of commited crimes '95": 68,
 "no. of commited crimes '96 ": 69,
 'client_id': 268,
 'birth_number': 262,
 'has disponent': 2}

In [15]:
np.bincount(y_train)


array([182,  32], dtype=int64)

In [16]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [17]:
np.bincount(y_res)


array([182, 182], dtype=int64)

# Testing models with default parameters


In [18]:
models = [
    SVC(probability=True),
    DecisionTreeClassifier(),
    MLPClassifier(),
]

In [19]:
model_names = [
    "Support Vector Machine",
    "         Decision Tree",
    "        Neural Network",
]

In [20]:
# Imbalanced Dataset

for model in models:
    model.fit(X_train, y_train)
    
for model, name in zip(models, model_names):
    print(name + ": {:.4f}%".format(model.score(X_test, y_test) * 100))
    y_pred = model.predict(X_test)
    # print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Support Vector Machine: 100.0000%
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        49
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

         Decision Tree: 100.0000%
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        49
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

        Neural Network: 98.1481%
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        49
         1.0       1.00      0.80      0.89         5

    accuracy                           0.98        54
   macro avg       0.99    

In [21]:
# Balanced Dataset

for model in models:
    model.fit(X_res, y_res)

In [22]:
for model, name in zip(models, model_names):
    print(name + ": {:.4f}%".format(model.score(X_test, y_test) * 100))
    y_pred_1 = model.predict(X_test)
    print(confusion_matrix(y_test, y_pred_1))
    print(classification_report(y_test, y_pred_1))
    # model.predict_proba(X_test)

Support Vector Machine: 100.0000%
[[49  0]
 [ 0  5]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        49
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

         Decision Tree: 100.0000%
[[49  0]
 [ 0  5]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        49
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

        Neural Network: 96.2963%
[[48  1]
 [ 1  4]]
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98        49
         1.0       0.80      0.80      0.80         5

    accuracy            